# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM, CuDNNLSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, dropout=0.2, name='lstm1'))
    model.add(CuDNNLSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm1'))
    model.add(Dropout(0.2))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])

#selected_indices = np.array(list(range(1,22)))

selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#scaler = StandardScaler()
scaler = MinMaxScaler()

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    model = RULmodel_LSTM(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model

In [6]:
num_features = len(selected_features)

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)

(30, 14)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

In [8]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0.01)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.01
Loading data from file and computing dataframes
not unrolled data
Printing shapes

Training data (X, y)
(17502, 30, 14)
(17502, 1)
Cross-Validation data (X, y)
(1, 30, 14)
(1, 1)
Testing data (X, y)
(100, 30, 14)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[[0.18373494 0.40680183 0.30975692 ... 0.33333333 0.71317829 0.7246617 ]
  [0.28313253 0.4530194  0.35263336 ... 0.33333333 0.66666667 0.73101353]
  [0.34337349 0.36952256 0.37052667 ... 0.16666667 0.62790698 0.62137531]
  ...
  [0.34337349 0.25724875 0.28494261 ... 0.16666667 0.6744186  0.53838719]
  [0.21084337 0.30063222 0.31634031 ... 0.41666667 0.6124031  0.64277824]
  [0.29819277 0.49008066 0.23345712 ... 0.16666667 0.70542636 0.71361502]]

 [[0.28313253 0.4530194  0.35263336 ... 0.33333333 0.66666667 0.73101353]
  [0.34337349 0.36952256 0.37052667 ... 0.16666667 0.62790698 0.62137531]
  [0.34337349 0.25615

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (tModel.data_handler.sequence_length, nFeatures)
model = get_compiled_model(shape, model_type='lstm')
tModel.change_model('ModelRUL_RNN_1', model, 'keras')

tModel.epochs = 100

tModel.train_model(learningRate_scheduler=lrate, verbose=1)

(30, 14)
training with cv
Train on 17502 samples, validate on 1 samples
Epoch 1/100
17502/17502 [==============================] - 6s 322us/step - loss: 8122.2965 - mean_squared_error: 8117.8252 - val_loss: 190.6811 - val_mean_squared_error: 186.0939
Epoch 2/100
17502/17502 [==============================] - 0s 16us/step - loss: 7525.3955 - mean_squared_error: 7520.5722 - val_loss: 76.6628 - val_mean_squared_error: 71.5152
Epoch 3/100
17502/17502 [==============================] - 0s 16us/step - loss: 6669.1226 - mean_squared_error: 6663.6022 - val_loss: 10.2004 - val_mean_squared_error: 4.2264
Epoch 4/100
17502/17502 [==============================] - 0s 16us/step - loss: 5752.7911 - mean_squared_error: 5746.3239 - val_loss: 36.2374 - val_mean_squared_error: 29.1885
Epoch 5/100
17502/17502 [==============================] - 0s 16us/step - loss: 4828.6305 - mean_squared_error: 4821.0123 - val_loss: 190.2532 - val_mean_squared_error: 181.9861
Epoch 6/100
17502/17502 [===================

17502/17502 [==============================] - 0s 9us/step - loss: 346.4079 - mean_squared_error: 322.7973 - val_loss: 92.6573 - val_mean_squared_error: 69.0426
Epoch 48/100
17502/17502 [==============================] - 0s 9us/step - loss: 331.1356 - mean_squared_error: 307.5196 - val_loss: 82.2957 - val_mean_squared_error: 58.7042
Epoch 49/100
17502/17502 [==============================] - 0s 9us/step - loss: 331.1792 - mean_squared_error: 307.5872 - val_loss: 76.4369 - val_mean_squared_error: 52.8411
Epoch 50/100
17502/17502 [==============================] - 0s 9us/step - loss: 326.6914 - mean_squared_error: 303.0872 - val_loss: 72.2386 - val_mean_squared_error: 48.6406
Epoch 51/100
17502/17502 [==============================] - 0s 9us/step - loss: 322.5242 - mean_squared_error: 298.9168 - val_loss: 82.2212 - val_mean_squared_error: 58.6225
Epoch 52/100
17502/17502 [==============================] - 0s 9us/step - loss: 324.0096 - mean_squared_error: 300.4155 - val_loss: 53.0816 - v

17502/17502 [==============================] - 0s 9us/step - loss: 276.7352 - mean_squared_error: 254.2893 - val_loss: 72.7628 - val_mean_squared_error: 50.3392
Epoch 95/100
17502/17502 [==============================] - 0s 9us/step - loss: 282.6712 - mean_squared_error: 260.2494 - val_loss: 51.7509 - val_mean_squared_error: 29.3495
Epoch 96/100
17502/17502 [==============================] - 0s 9us/step - loss: 281.0964 - mean_squared_error: 258.7306 - val_loss: 52.4160 - val_mean_squared_error: 30.0488
Epoch 97/100
17502/17502 [==============================] - 0s 9us/step - loss: 274.7689 - mean_squared_error: 252.4185 - val_loss: 68.4766 - val_mean_squared_error: 46.1481
Epoch 98/100
17502/17502 [==============================] - 0s 9us/step - loss: 283.2804 - mean_squared_error: 260.9890 - val_loss: 57.1001 - val_mean_squared_error: 34.8317
Epoch 99/100
17502/17502 [==============================] - 0s 9us/step - loss: 269.2520 - mean_squared_error: 246.9877 - val_loss: 56.1758 - v

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)

#print(tModel.y_predicted)
#print(tModel.y_predicted_rounded)
#print(tModel.y_test)

for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 69us/step
scores
Engine 1, Predicted RUL 114.54479217529297, Rounded RUL 114.0, Real RUL 112.0
Engine 2, Predicted RUL 119.43768310546875, Rounded RUL 119.0, Real RUL 98.0
Engine 3, Predicted RUL 52.30144500732422, Rounded RUL 52.0, Real RUL 69.0
Engine 4, Predicted RUL 97.2656478881836, Rounded RUL 97.0, Real RUL 82.0
Engine 5, Predicted RUL 108.40982055664062, Rounded RUL 108.0, Real RUL 91.0
Engine 6, Predicted RUL 112.38052368164062, Rounded RUL 112.0, Real RUL 93.0
Engine 7, Predicted RUL 109.14957427978516, Rounded RUL 109.0, Real RUL 91.0
Engine 8, Predicted RUL 99.06985473632812, Rounded RUL 99.0, Real RUL 95.0
Engine 9, Predicted RUL 120.26200866699219, Rounded RUL 120.0, Real RUL 111.0
Engine 10, Predicted RUL 91.28961181640625, Rounded RUL 91.0, Real RUL 96.0
Engine 11, Predicted RUL 91.74349212646484, Rounded RUL 91.0, Real RUL 97.0
Engine 12, Predicted RUL 106.9220199584961, Rounded RUL 106.0, Real RUL 124.0
Engine 13, Predicte